In [37]:
from ibm_watsonx_ai import APIClient
from ibm_watsonx_ai import Credentials
from ibm_watsonx_ai.foundation_models import ModelInference
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.foundation_models.utils.enums import DecodingMethods
from dotenv import load_dotenv
import os

load_dotenv()  # take environment variables

credentials = Credentials(
                   url = os.getenv('IBMLOCATION'),
                   api_key = os.getenv('APIKEY')
                  )

project_id = os.getenv('PROJECTID')
space_id = os.getenv('SPACEID')

client = APIClient(credentials, project_id = project_id)

client.spaces.list(limit=10)
client.set.default_project(project_id)

Cannot set Project or Space
Reason: {"id":"WSCPA0000E","code":404,"error":"Not Found","reason":"Failed to retrieve project: 09208850-977d-444e-aa03-e3bf3dabbcd2. Failed to retrieve Cloudant document: 09208850-977d-444e-aa03-e3bf3dabbcd2. not_found: missing.","message":"Resource requested by the client was not found."}


CannotSetProjectOrSpace: Cannot set Project or Space
Reason: {"id":"WSCPA0000E","code":404,"error":"Not Found","reason":"Failed to retrieve project: 09208850-977d-444e-aa03-e3bf3dabbcd2. Failed to retrieve Cloudant document: 09208850-977d-444e-aa03-e3bf3dabbcd2. not_found: missing.","message":"Resource requested by the client was not found."}

In [29]:
gen_parms = {
    GenParams.DECODING_METHOD: DecodingMethods.SAMPLE,
    GenParams.MAX_NEW_TOKENS: 1000
}
model_id = client.foundation_models.TextModels.GRANITE_3_8B_INSTRUCT
verify = False

model = ModelInference(
  model_id=model_id,
  credentials=credentials,
  params=gen_parms,
  project_id=project_id,
  space_id=space_id,
  verify=verify,
)


In [34]:
import docx2txt
# extract text
text = docx2txt.process("nda.docx")
#print(text)


prompt_txt = "As a legal assistant, find not needed sentences in the following text. Output the first not needed or highly useless sentences and why " + text

In [35]:
generated_text_response = model.generate_text(prompt=prompt_txt, params=gen_parms)

print("Output from generate_text() method:")
print(generated_text_response)

Output from generate_text() method:
.	If you decide that you do not wish to proceed with the Transaction, you will promptly inform the Company of that decision.  In that case, or at any time upon the request of the Company for any reason, you will promptly (a) deliver to the Company all Evaluation Material furnished to you or your Representatives by or on behalf of the Company (and all copies thereof whether received from the Company or made by you or your Representatives) and (b) either deliver to the Company or destroy all materials prepared by you or your Representatives which constitute Evaluation Material without retaining a copy of any such material, with any such destruction certified to the Company in writing by your authorized officer supervising such destruction.

This sentence is not necessary because it is repeated later in the agreement with the same information. Both sections appear under the same heading and provide identical instructions for returning or destroying the 